### Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score

from utils.loader import llm, emb_fn, strings
from utils.utils import prepare_data, set_all_seeds, save_predictions
from utils.optimizer import Protegi

In [2]:
set_all_seeds(42)

### Data

In [2]:
prepare_data('spam', '/home/umbilnm/python_ml/AutomatizationPromptEngeneering/data/spam.csv', [100, 1000]) 
df = pd.read_csv('/home/umbilnm/python_ml/AutomatizationPromptEngeneering/data/spam_100.csv').drop(
    columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4']
)
messages = df['v2']

NameError: name 'prepare_data' is not defined

### Model

In [4]:
template = strings['templates']['prediction_template']
template = PromptTemplate(template=template, input_variables=['message'])

preds = []
for message in tqdm(messages):
    prediction = llm.invoke(template.format(message=message)).content
    preds.append(prediction)
    df['predictions'] = 'ham' if prediction=='No' else 'spam'

In [7]:
df['predictions'] = preds
df['predictions'] = df['predictions'].apply(lambda x: 0 if x=='No' else 1)
df['v1'] = df['v1'].apply(lambda x: 0 if x=='ham' else 1)

NameError: name 'preds' is not defined

In [5]:
preds_df = pd.read_csv('data/predistions/first_predictions.csv')
df = pd.read_csv('data/spam_100.csv').drop(columns=[f'Unnamed: {i}' for i in range(2, 5)])
df['preds'] = preds_df['predictions']
df['v1'] = df['v1'].apply(lambda x: 1 if x=='spam' else 0)

### Metrics

In [66]:
accuracy = accuracy_score(df['v1'], df['predictions'])
recall = recall_score(df['v1'], df['predictions'])
f1 = f1_score(df['v1'], df['predictions'])
precision = precision_score(df['v1'], df['predictions'])
print(f'Accuracy = {accuracy:.2f}\nRecall = {recall:.2f}\nPrecision = {precision:.2f}\nF1 = {f1:.2f}')

Accuracy = 0.82
Recall = 1.00
Precision = 0.40
F1 = 0.57


### Protegi

In [3]:
preds_df = pd.read_csv('data/predistions/first_predictions.csv')
df = pd.read_csv('data/spam_100.csv').drop(columns=[f'Unnamed: {i}' for i in range(2, 5)])
prot = Protegi()
prompt = strings['templates']['prediction_template']
sample= df.sample(20)

In [4]:
mc_sampled_task_sections, new_task_sections =prot.expand_candidates(prompts=[prompt], df=sample, batch_size=10)

In [5]:
mc_sampled_task_sections

[('content', 'Determine if the following message is spam or not.'),
 ('additional_kwargs', {}),
 ('response_metadata',
  {'token_usage': {'completion_tokens': 11,
    'prompt_tokens': 39,
    'total_tokens': 50},
   'model_name': 'gpt-3.5-turbo-1106',
   'system_fingerprint': 'fp_77a673219d',
   'finish_reason': 'stop',
   'logprobs': None}),
 ('type', 'ai'),
 ('name', None),
 ('id', 'run-3387615a-9673-4a3e-81bd-50b5f66e4c93-0'),
 ('example', False),
 ('tool_calls', []),
 ('invalid_tool_calls', []),
 ('content',
  'Can you confirm if the message below is spam?\nMessage: {message}'),
 ('additional_kwargs', {}),
 ('response_metadata',
  {'token_usage': {'completion_tokens': 15,
    'prompt_tokens': 35,
    'total_tokens': 50},
   'model_name': 'gpt-3.5-turbo-1106',
   'system_fingerprint': 'fp_77a673219d',
   'finish_reason': 'stop',
   'logprobs': None}),
 ('type', 'ai'),
 ('name', None),
 ('id', 'run-19109500-ea72-4b66-a3ac-38c7dc6af444-0'),
 ('example', False),
 ('tool_calls', []),
 (